In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense
# from sklearn.preprocessing import MinMaxScaler
# import warnings
# warnings.filterwarnings("ignore", category=FutureWarning)
# warnings.filterwarnings("ignore", category=RuntimeWarning)
np.random.seed(0)

In [7]:
lastfm_360_behav = pd.read_csv('lastfm-dataset-360K/behav-360k-processed.csv')
uniq_users = lastfm_360_behav['user_email'].unique()
upd_ids = dict((id_, i) for i, id_ in enumerate(uniq_users))
lastfm_360_behav['user_email'] = lastfm_360_behav['user_email'].map(lambda x: upd_ids[x])

In [8]:
ratings = lastfm_360_behav[['user_email', 'artist_id', 'std_plays']]

In [9]:
artists = ratings['artist_id'].unique()
artists_num = artists.shape[0] 

In [131]:
def user_split(ratings_, test_frac=0.1):
    users = ratings_['user_email'].unique()
    np.random.shuffle(users)
    train_size = int((1 - test_frac) * users.shape[0])
    train_users, test_users = users[:train_size], users[train_size:]
    return train_users, test_users
# ratings_[ratings_['user_email'].isin(train_users)].reset_index(drop=True), \
#            ratings_[ratings_['user_email'].isin(test_users)].reset_index(drop=True)

def holdout_artist(test_users, ratings_, k=5):
    test = ratings_[ratings_['user_email'].isin(test_users)].reset_index(drop=True)
    large_groups = test.groupby('user_email').filter(lambda x: x['user_email'].count() >= 2 * k)
    holdout = large_groups.groupby('user_email').sample(k)    
    return holdout

# maybe median
def user_artist_matrix(data, artists, average=True):
    users = data['user_email'].unique()
    user_artist = pd.DataFrame(-np.ones((users.shape[0], artists.shape[0])), columns=artists, index=users)
    mask = pd.DataFrame(np.zeros((users.shape[0], artists.shape[0]), dtype=bool), columns=artists, index=users)
    for user in users:
        user_frame = data[data['user_email'] == user]
        if average:
            user_artist.loc[user] = user_frame['std_plays'].mean()
        user_artist.loc[user, user_frame['artist_id']] = user_frame['std_plays'].to_numpy()
        mask.loc[user, user_frame['artist_id']] = True
    return user_artist, mask

def masked_mse(y_true, y_pred):
    pass

In [113]:
train_users, test_users = user_split(ratings, test_frac=0.1)

In [114]:
holdout = holdout_artist(test_users, ratings, k=5)

In [115]:
test_user_artist_matrix, mask = user_artist_matrix(test.iloc[:100], artists) #can work only for batches

In [ ]:
# def create_autorec(input_dim, hidden_dim):
#      # User branch
#     inputs = x = tf.keras.layers.Input(shape=input_dim, name='user_rating')
#     x = Dense(500, activation='', name='LatentSpace', kernel_regularizer=regularizers.l2(reg))(x)
#     output_layer = Dense(X.shape[1], activation=last_activation, name='UserScorePred', kernel_regularizer=regularizers.l2(reg))(x)

#     # Model definition
#     model = tf.keras.models.Model(inputs=[user_id, item_id], outputs=[vectors_product], name='shallow_model')
#     return model